In [ ]:
from sympy import cos, sin, sqrt, pi, Matrix, Symbol, exp, print_latex, simplify
#from sympy.physics.quantum import TensorProduct, Dagger
import numpy as np
from numpy import linspace
#import matplotlib.pyplot as plt
#import math
from theoric.tools import *
#import torch
from torch import tensor

theta = Symbol('theta',real=True)
phi = Symbol('phi',real=True)
gamma = Symbol('gamma',real=True, positive=True)
p = Symbol('p',real=True, positive=True)


# tem que arrumar a função abaixo ainda
def calculated_rho_AB(theta, phi, p):
    state = Matrix([[(cos(theta/2)),
                    (sqrt(p)),
                    (sqrt(1-p)*exp(1j*phi)*sin(theta/2)),
                    0]])
    M_numpy = np.array(state.tolist(), dtype=np.complex64)
    return M_numpy
    #return state

def get_target_op(func_estado):
    state1 = simplify(func_estado)
    state2 = np.zeros(4,dtype=complex)
    aux = 0
    for i in state1[0]:
        state2[aux] = i
        aux += 1
    target_op = np.outer(state2.conj(), state2)
    target_op = tensor(target_op)
    return target_op

state = calculated_rho_AB(pi/2, 0, 0.5)
state

In [ ]:
a = get_target_op(calculated_rho_AB(pi/2, 0, 1))
a


In [ ]:
'''                                  ReadMe                                                      '''
'''# Nessa célula vamos apenas mudar a função geradora do estado a ser preparado                 '''
'''# Basta alterar a função que gera o estado, nesse caso target_op = init_state_ref(n_qubits, p)'''

from src.vqa_tools import general_vqacircuit_penny 
from src.generalize import *
from torch import tensor
from numpy import sin,cos,sqrt,outer,zeros, pi
import cmath
import pickle
from theoric.ad_theoric import *

def run_calcs(list_p, epochs):
    coerencias_R = []
    coerencias_L = []
    pretrain = True
    count = 0
    n_qubits = 2
    depht = 3
    _, params, _, _ = start_things(n_qubits, depht)
    for p in list_p:
        print(f'{count} de {len(list_p)}')
        count += 1
        #_, params, _, _ = start_things(n_qubits, depht)
        
        circuit, _ = general_vqacircuit_penny(params, n_qubits, depht)

        # defina o estado a ser preparado abaixo
        #------------------------------------------------------------
        #target_op = bpf(pi/2, 0, p)
        target_op = get_target_op(calculated_rho_AB(pi/2, 0, p))
        #------------------------------------------------------------
    
        qc, qr, params = optmize(epochs, n_qubits, circuit, params, target_op, pretrain, 120)
        pretrain = False
        rho = tomograph(qc, qr)
        print(rho)
        coerencias_L, coerencias_R = results(rho, coerencias_R, coerencias_L)
    mylist = [coerencias_L, coerencias_R]
    with open('data/ad/ad-coherences.pkl', 'wb') as f:
        pickle.dump(mylist, f)
    with open('data/ad/ad-best_params.pkl', 'wb') as f:
        pickle.dump(params, f)
    plot_theoric_ad(list_p)
    plots(list_p, coerencias_R, coerencias_L)
    #save = [list_p, coerencias_R, coerencias_L]
    #with open('data/BPFlist_p-coerencias_R-coerencias_L.pkl', 'wb') as f:
    #    pickle.dump(save, f)
    

In [ ]:
# teste
list_p = np.linspace(0,1,4)
run_calcs(list_p, 2)

In [ ]:
list_p = np.linspace(0,1,21)
run_calcs(list_p, 85)